In [1]:
import spacy

import numpy as np
import en_core_web_sm
nlp = en_core_web_sm.load()

from spacy.lang.en.stop_words import STOP_WORDS

import xml.etree.ElementTree as ET
n_topics=50
def get_titles(file):
    tree = ET.parse(file)
    root = tree.getroot()
    buffer = []
    i=1
    for title in root.iter('title'):
        if i<=int(n_topics):
            buffer.append(title.text.strip())
        i=i+1
    return buffer
file="topics-task-2.xml"
topics=get_titles(file)

# using sense2vec spacy3

In [5]:
from sense2vec import Sense2VecComponent
s2v = nlp.add_pipe("sense2vec")
s2v.from_disk("s2v_reddit_2015_md/s2v_old")

In [6]:
#DEMO: https://explosion.ai/demos/sense2vec?word=Canon&sense=auto&model=2015

from sense2vec import Sense2Vec
standalone_s2v = Sense2Vec().from_disk("./s2v_reddit_2015_md/s2v_old")
'''
query = "Google|ORG"
assert query in s2v_
most_similar = s2v_.most_similar(query, n=3)
most_similar
'''

'\nquery = "Google|ORG"\nassert query in s2v_\nmost_similar = s2v_.most_similar(query, n=3)\nmost_similar\n'

using sense2vec standard-usage

In [7]:
'''
from spacy import displacy

text = "When Sebastian Thrun started working on self-driving cars at Google in 2007, few people outside of the company took him seriously."

nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
displacy.serve(doc, style="ent")
'''

'\nfrom spacy import displacy\n\ntext = "When Sebastian Thrun started working on self-driving cars at Google in 2007, few people outside of the company took him seriously."\n\nnlp = spacy.load("en_core_web_sm")\ndoc = nlp(text)\ndisplacy.serve(doc, style="ent")\n'

# 
- spacy (token,spacy_sense) (Satz)
- token.other_senses in Wörter
- if spacy_sense nicht [all_senses]: 

- Google - PROPN
- Sense2Vec - Google|ORG, 
- Welche Kombination von Google gibt es - nehmen ein (z.B. ORG sub von PROPN)

In [7]:
def get_similar_words(word, topn):
    ms = nlp.vocab.vectors.most_similar(np.asarray([nlp.vocab.vectors[nlp.vocab.strings[word]]]), n=topn)
    words = [nlp.vocab.strings[w] for w in ms[0][0]]
    return words

def similar_words_with_wordembeddings(topic, topn):
    tags = ['CD',
            'JJ',
            'RB', 
            'NN', 'NNS','NNP','NNPS', 
            'VB']
    pos_tags = ['PROPN','VERB','NOUN','NUM']

    doc = nlp(topic)
    similar_words={}
    for token in doc:
        top_similar_words=[]
        if token.tag_ in tags or token.pos_ in pos_tags:
            if token.lemma_ not in STOP_WORDS or token.text not in STOP_WORDS:
                print(token.text)
                try:
                    similar_words_by_wordembeddings = get_similar_words(token.text, topn)
                    
                    for word in similar_words_by_wordembeddings:
                        if (word != token.text) and (nlp(word)[0].lemma_ != token.lemma_):
                            top_similar_words.append(nlp(word)[0].lemma_.lower())
                except ValueError as err:
                    print(err)
                print(set(top_similar_words))
                similar_words[token.text]=list(set(top_similar_words))
                print("="*80)
    return similar_words
    
def get_similars_sense2vec(topic, topn):
    tags = ['CD',
            'JJ',
            'RB', 
            'NN', 'NNS','NNP','NNPS', 
            'VB']
    pos_tags = ['PROPN','VERB','NOUN','NUM']

    doc = nlp(topic)
    similar_words={}
    for token in doc:
        top_similar_words=[]
        if token.tag_ in tags or token.pos_ in pos_tags:
            if token.lemma_ not in STOP_WORDS or token.text not in STOP_WORDS:
                try:
                    for e in token._.s2v_most_similar(topn):
                        word = e[0][0].strip() #get only word from ((word, tag), proba)
                        if (word != token.text) and (nlp(word)[0].lemma_ != token.lemma_):
                            top_similar_words.append(word)
                except ValueError as err:
                    for ent in doc.ents:
                        if ent.text == token.text:
                            try:
                                for e in ent._.s2v_most_similar(topn):
                                    word = e[0][0].strip() #get only word from ((word, tag), proba)
                                    if (word != token.text) and (nlp(word)[0].lemma_ != token.lemma_):
                                        top_similar_words.append(word)

                            except ValueError as err:
                                print(token._.s2v_other_senses == ent._.s2v_other_senses)
                                query = token._.s2v_other_senses[0] #get first similar words by entity_tag
                                print(token._.s2v_other_senses[0])
                                print(ent._.s2v_other_senses[0])
                                for e in standalone_s2v.most_similar(query, n=topn):
                                    word = e[0].split("|")[0].strip() #get only word from (word|tag, proba)
                                    if (word != token.text) and (nlp(word)[0].lemma_ != token.lemma_):
                                        top_similar_words.append(word)
                print(set(top_similar_words))
                similar_words[token.text]=list(set(top_similar_words))
                print("="*80)
    return similar_words

In [8]:
topics[0]

'What is the difference between sex and love?'

In [9]:
def similarwords_replace(query, similar_words):
    import copy
    
    expanded_queries=[]
    new=[]
    
    
    
    for word, similars in similar_words.items():
        for similar_word in similars:
            print(similar_word)
            new_query=copy.deepcopy(query).replace(word, similar_word)
            expanded_queries.append(new_query)
    
    keys = similar_words.keys()
    for key in keys:
        for other_key in keys:
            if other_key!=key:
                for sw in similar_words[key]:
                    pos_key = [ i for i in range(len(query.split(" "))) if query.split(" ")[i]==key]
                    #print(pos_key)
                    n = copy.deepcopy(query).replace(key, sw)
                    pos = [i+len(sw.split(" "))-1 for i in pos_key]
                    #print(pos)
                    #laptop 1 - desktop PC 1,2
                    for p in pos_key:
                        pos.append(p)
                    pos = list(set(pos))
                    print(pos)
                    new.append(n)
                    for swo in similar_words[other_key]:
                        pos_other = [ i for i in range(len(n.split(" "))) if n.split(" ")[i]==other_key and i not in pos]
                        print(pos_other)
                        print("========")
                        import numpy
                        text_as_arr = numpy.array(n.split(" "))
                        for idx in pos_other:
                            text_as_arr[idx]=swo
                        #nn = copy.deepcopy(n).replace(other_key, swo)
                        nn = " ".join(text_as_arr)
                        new.append(nn)
                    print("=======================================================")
    return new, expanded_queries

In [10]:
query=topics[0]
key="laptop"
[ i for i in range(len(query.split(" "))) if query.split(" ")[i]==key]

[]

In [11]:
#problem desktop PC replaced for laptop
# desktop replace also from "desktop PC"

In [13]:
r = get_similars_sense2vec(topics[0], topn=2)
print(r)

{'actual difference', 'real difference'}
{'only sex', 'actual sex'}
{'real love'}
{'difference': ['actual difference', 'real difference'], 'sex': ['only sex', 'actual sex'], 'love': ['real love']}


In [15]:
a,b=similarwords_replace(topics[0], r)
print(topics[0])
print(a)
print(b)

actual difference
real difference
only sex
actual sex
real love
[3, 4]
[6]
[6]
[3, 4]
[6]
[6]
[3, 4]
[]
[3, 4]
[]
[5, 6]
[3]
[3]
[5, 6]
[3]
[3]
[5, 6]
[]
[5, 6]
[]
[]
[3]
[3]
[]
[5]
[5]
What is the difference between sex and love?
['What is the actual difference between sex and love?', 'What is the actual difference between only sex and love?', 'What is the actual difference between actual sex and love?', 'What is the real difference between sex and love?', 'What is the real difference between only sex and love?', 'What is the real difference between actual sex and love?', 'What is the actual difference between sex and love?', 'What is the actual difference between sex and love?', 'What is the real difference between sex and love?', 'What is the real difference between sex and love?', 'What is the difference between only sex and love?', 'What is the actual dif between only sex and love?', 'What is the real diffe between only sex and love?', 'What is the difference between actual sex an

In [16]:
a

['What is the actual difference between sex and love?',
 'What is the actual difference between only sex and love?',
 'What is the actual difference between actual sex and love?',
 'What is the real difference between sex and love?',
 'What is the real difference between only sex and love?',
 'What is the real difference between actual sex and love?',
 'What is the actual difference between sex and love?',
 'What is the actual difference between sex and love?',
 'What is the real difference between sex and love?',
 'What is the real difference between sex and love?',
 'What is the difference between only sex and love?',
 'What is the actual dif between only sex and love?',
 'What is the real diffe between only sex and love?',
 'What is the difference between actual sex and love?',
 'What is the actual dif between actual sex and love?',
 'What is the real diffe between actual sex and love?',
 'What is the difference between only sex and love?',
 'What is the difference between only sex 

In [17]:
b

['What is the actual difference between sex and love?',
 'What is the real difference between sex and love?',
 'What is the difference between only sex and love?',
 'What is the difference between actual sex and love?',
 'What is the difference between sex and real love?']

In [20]:
rr = similar_words_with_wordembeddings(topics[0], 2)
print(rr)
similarwords_replace(topics[0], rr)

difference
{'differance'}
sex
set()
love
set()
{'difference': ['differance'], 'sex': [], 'love': []}
differance
[3]
[3]


(['What is the differance between sex and love?',
  'What is the differance between sex and love?'],
 ['What is the differance between sex and love?'])

In [30]:
similar_words_by_topics=[]
for topic in topics:
    print(topic)
    similar_words_by_topics.append(get_similars_sense2vec(topic, topn=5))

What is the difference between sex and love?
{'big difference', 'massive difference', 'large difference', 'actual difference', 'real difference'}
{'actual sex', 'sexual partner', 'just sex', 'only sex'}
{'deep love', 'just love', 'true love', 'real love'}
Which is better, a laptop or a desktop?
{'desktop pc', 'netbook', 'desktop computer', 'computer', 'desktop PC'}
{'Windows desktop', 'chromebook', 'Chromebook'}
Which is better, Canon or Nikon?
True
Canon|ORG
Canon|ORG
{'6D', 'Nikon', 'Pentax', '7D', 'Minolta'}
{'Canon', 'Pentax', 'Minolta', 'D750', '60D'}
What are the best dish detergents?
{'sauces', 'seasoning', 'soup', 'noodles', 'sauce'}
{'shampoos', 'sulfates', 'surfactants', 'oils', 'harsh chemicals'}
What are the best cities to live in?
{'small cities', 'many cities', 'large cities', 'towns', 'major cities'}
set()
What is the longest river in the U.S.?
{'creek', 'lake', 'shoreline', 'shore', 'small river'}
{'US', 'USA.', 'US.', 'USA', 'U.S'}
Which is healthiest: coffee, green te

{'petroleum jelly', 'thin layer', 'cotton ball', 'goop'}
Which is better, Linux or Microsoft?
set()
{'MSFT', 'microsoft', 'MS', 'Windows'}
Which is better, Pepsi or Coke?
{'Dr Pepper', 'Coca-Cola', 'Coke', 'RC Cola'}
{'Diet Coke', 'Pepsi', 'RC Cola', 'Dr Pepper', 'Dr. Pepper'}
What is better, Google search or Yahoo search?
set()
set()
{'yahoo', 'Google'}
set()
Which one is better, Netflix or Blockbuster?
set()
set()
Which browser is better, Internet Explorer or Firefox?
{'Google Chrome', 'web browser', 'Chrome'}
set()
set()
set()
Which is a better vehicle: BMW or Audi?
{'own vehicle', 'said car', 'other vehicle', 'passenger', 'car'}
{'Porsche', 'Toyota', 'Lexus', 'Audi'}
{'Toyota', 'Infiniti', 'Porsche', 'BMW', 'Lexus'}
Which one is better, an electric stove or a gas stove?
{'Electric', 'electricity'}
{'hot plate', 'gas stove', 'microwave', 'stovetop'}
{'diesel', 'petrol', 'fuel', 'gasoline'}
{'hot plate', 'gas stove', 'microwave', 'stovetop'}
What planes are best, Boeing or Airbus?
{'

{'entire code', 'other code', 'only code', 'new code', 'actual code'}
{'unreadable', 'legible', 'illegible', 'editable', 'verbose'}
{'entire code', 'other code', 'only code', 'new code', 'actual code'}
{'writen'}
True
Scala|ORG
Scala|ORG
{'Haskell', 'Clojure'}
Which operating system has better performance: Windows 7 or Windows 8?
{'systems', 'enterprise', 'operations', 'operation'}
{'whole system', 'other system', 'actual system', 'entire system'}
{'performace', 'overall performance', 'preformance', 'perfomance', 'actual performance'}
set()
{'11', '9', '8', '10', '6'}
set()
{'9', '7', '6', '10'}
Which smartphone has a better battery life: Xperia or iPhone?
{'tablet', 'iPad', 'smart phone', 'mobile phone', 'Android phone'}
{'internal battery', 'small battery', 'lithium battery', 'good battery'}
{'own life', 'future life', 'life-', 'current life'}
{'iPad', 'Android phones', 'Android', 'iPhone', 'Sony phones'}
{'iPad', 'Android phones', 'other phones', 'android phones'}
Which four wheel t

In [38]:
similar_words_by_topics[0]['difference']

['big difference',
 'massive difference',
 'large difference',
 'actual difference',
 'real difference']

In [33]:
similar_words_wordembeddings=[]
for topic in topics:
    print(topic)
    similar_words_wordembeddings.append(similar_words_with_wordembeddings(topic, topn=5))

What is the difference between sex and love?
difference
{'differance', 'mean'}
sex
{'porn', 'lesbia', 'smut'}
love
set()
Which is better, a laptop or a desktop?
laptop
{'portable', 'lap', 'notebooks', 'computer'}
desktop
{'desk', 'destop', 'computer'}
Which is better, Canon or Nikon?
Canon
{'canon', 'konica', 'cannon', 'smoothbore', 'ae-1'}
Nikon
{'ericsson', 'canon', 'konica', 'ae-1', 'mpix'}
What are the best dish detergents?
dish
{'sauce', 'restaurant', 'main', 'vichyssoise'}
detergents
{'dispersant', 'polychlorinated', 'thirdhand', 'chlorofluorocarbon', 'additive'}
What are the best cities to live in?
cities
{'township', 'metropolis'}
live
{'livng'}
What is the longest river in the U.S.?
river
{'fording', 'creek', 'waterway'}
U.S.
{'u.s', 'kirkuk', 'america', 'histroy'}
Which is healthiest: coffee, green tea or black tea and why?
coffee
{'tea', 'coffees', 'capuccino', 'teabag'}
green
{'red', 'greeen'}
tea
{'coffee', 'libation', 'coffees', 'teabag'}
black
{'white', 'blck'}
tea
{'cof

{'bristle', 'handkerchief', 'dauber', 'glues', 'dacron'}
Which is better, Linux or Microsoft?
Linux
{'ppa', 'ubuntu', 'vmware', 'backtrack'}
Microsoft
{'acrobat', 'xp', 'nt4', 'microsofts'}
Which is better, Pepsi or Coke?
Pepsi
{'anheuser', 'cola', 'coke', 'soft', 'cokes'}
Coke
{'anheuser', 'seltzer', 'coke', 'soft', 'cokes'}
What is better, Google search or Yahoo search?
Google
{'facebook', 'altavista', 'streetview', 'foursquare'}
search
{'searches', 'searcher', 'find'}
Yahoo
{'messinger', 'altavista', 'google', 'streetview'}
search
{'searches', 'searcher', 'find'}
Which one is better, Netflix or Blockbuster?
Netflix
{'comcast', 'nflx', 'ameritech', 'joost'}
Blockbuster
{'two', 're', 'smash', 'fantasy', 'big'}
Which browser is better, Internet Explorer or Firefox?
browser
{'pagination', 'text', 'firefox', 'foxfire', 'webbrowser'}
Internet
{'internet', 'online', 'intranet', 'intenet', 'web'}
Explorer
{'firefox', 'xp', 'foxfire', 'explorer'}
Firefox
{'webbrowser', 'backtrack', 'foxfire'

{'dissimilarity', 'differance', 'diverge', 'multiplicity'}
MySQL
{'erd', 'query', 'dbi', 'php4', 'adf'}
PostgreSQL
{'erd', 'query', 'dbi', 'php4', 'adf'}
performance
{'perfomance', 'performace', 'perfrom', 'rehearse'}
Is Java code more readable than code written in Scala?
Java
{'pagination', 'rmi', 'javascript'}
code
{'encoding', 'color', '3166'}
readable
{'retrievable', 'text', 'paged', '40'}
code
{'encoding', 'color', '3166'}
written
{'composing', 'authored'}
Scala
{'data', 'python', 'pyside', 'awt', 'linear'}
Which operating system has better performance: Windows 7 or Windows 8?
operating
{'operate', 'non', 'operation'}
system
{'sytems', 'sytem', 'controll'}
performance
{'perfomance', 'performace', 'perfrom', 'rehearse'}
Windows
{'windo', 'xp', 'nt4', 'pop'}
7
{'5', '9', '8', '6'}
Windows
{'windo', 'xp', 'nt4', 'pop'}
8
{'9', '7', '11', '6'}
Which smartphone has a better battery life: Xperia or iPhone?
smartphone
{'marketshare', 'handset', 'mobile'}
battery
{'lap', 'lead', 'batt', '

{'b', 'soccer', 'football', 'gridiron'}
soccer
{'b', 'kickball', 'football', 'gridiron'}
players
{'play'}


In [34]:
similar_words_wordembeddings[0]['difference']

['differance', 'mean']

In [43]:
import itertools
import copy

def topics_and_similar_words(similar_words_by_topics):
    global topic
    shorted_topics =[]
    expanded_topics=[]
    similar_words_by_topics_copy = copy.deepcopy(similar_words_by_topics)
    index = 0
    for similar_words in similar_words_by_topics_copy:
        #print(similar_words)
        topic = topics[index]
        tmp = []
        for key,value_ in similar_words.items():
            value = copy.deepcopy(value_)
            value.insert(0,key)
            tmp.append(list(set(value)))
        #print(tmp)
        shorted_merged = list(set(list(itertools.chain.from_iterable(tmp))))
        shorted_topic = " ".join(list(set(" ".join(shorted_merged).split(" ")))) #after merging remove the same words in a query
        shorted_topics.append(shorted_topic)

        #expansion
        #expanded_merged = shorted_topic and then: topic+...
        print(topic.split(" "))
        print(list(set(shorted_topic.split(" "))))

        expanded_merged=" ".join([w for w in list(set(shorted_topic.split(" "))) if w not in topic])
        print(expanded_merged)
        expanded_topic = topic + " " + expanded_merged
        print(expanded_topic)
        expanded_topics.append(expanded_topic)
        index = index +1
    return shorted_topics, expanded_topics

In [44]:
shorted_topics, expanded_topics = topics_and_similar_words(similar_words_by_topics)
import pandas as pd
df = pd.DataFrame({"title": topics, "5_similar":similar_words_by_topics, "shorted_topic": shorted_topics, "expanded_topic": expanded_topics})
df.index +=1

df[['title','expanded_topic']]

['What', 'is', 'the', 'difference', 'between', 'sex', 'and', 'love?']
['real', 'sex', 'large', 'deep', 'true', 'only', 'just', 'love', 'actual', 'partner', 'massive', 'difference', 'big', 'sexual']
real large deep true only just actual partner massive big sexual
What is the difference between sex and love? real large deep true only just actual partner massive big sexual
['Which', 'is', 'better,', 'a', 'laptop', 'or', 'a', 'desktop?']
['PC', 'chromebook', 'netbook', 'desktop', 'Windows', 'computer', 'Chromebook', 'pc', 'laptop']
PC chromebook netbook Windows computer Chromebook pc
Which is better, a laptop or a desktop? PC chromebook netbook Windows computer Chromebook pc
['Which', 'is', 'better,', 'Canon', 'or', 'Nikon?']
['Canon', '6D', 'Nikon', 'Pentax', '7D', 'Minolta', 'D750', '60D']
6D Pentax 7D Minolta D750 60D
Which is better, Canon or Nikon? 6D Pentax 7D Minolta D750 60D
['What', 'are', 'the', 'best', 'dish', 'detergents?']
['chemicals', 'dish', 'sauces', 'harsh', 'sulfates', '

,title,expanded_topic
1,What is the difference between sex and love?,What is the difference between sex and love? r...
2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? PC chr..."
3,"Which is better, Canon or Nikon?","Which is better, Canon or Nikon? 6D Pentax 7D ..."
4,What are the best dish detergents?,What are the best dish detergents? chemicals s...
5,What are the best cities to live in?,What are the best cities to live in? many larg...
6,What is the longest river in the U.S.?,What is the longest river in the U.S.? US USA....
7,"Which is healthiest: coffee, green tea or blac...","Which is healthiest: coffee, green tea or blac..."
8,What are the advantages and disadvantages of P...,What are the advantages and disadvantages of P...
9,Why is Linux better than Windows?,Why is Linux better than Windows?
10,How to sleep better?,How to sleep better? wake bed nap


In [45]:
shorted_topics, expanded_topics = topics_and_similar_words(similar_words_wordembeddings)
import pandas as pd
df = pd.DataFrame({"title": topics, "5_similar":similar_words_by_topics, "shorted_topic": shorted_topics, "expanded_topic": expanded_topics})
df.index +=1
df[['title','expanded_topic']]

['What', 'is', 'the', 'difference', 'between', 'sex', 'and', 'love?']
['sex', 'lesbia', 'love', 'differance', 'mean', 'difference', 'porn', 'smut']
lesbia differance mean porn smut
What is the difference between sex and love? lesbia differance mean porn smut
['Which', 'is', 'better,', 'a', 'laptop', 'or', 'a', 'desktop?']
['lap', 'portable', 'desk', 'desktop', 'computer', 'laptop', 'destop', 'notebooks']
portable computer destop notebooks
Which is better, a laptop or a desktop? portable computer destop notebooks
['Which', 'is', 'better,', 'Canon', 'or', 'Nikon?']
['ericsson', 'Canon', 'canon', 'Nikon', 'konica', 'cannon', 'smoothbore', 'ae-1', 'mpix']
ericsson canon konica cannon smoothbore ae-1 mpix
Which is better, Canon or Nikon? ericsson canon konica cannon smoothbore ae-1 mpix
['What', 'are', 'the', 'best', 'dish', 'detergents?']
['dispersant', 'dish', 'detergents', 'polychlorinated', 'thirdhand', 'restaurant', 'chlorofluorocarbon', 'main', 'sauce', 'additive', 'vichyssoise']
disp

,title,expanded_topic
1,What is the difference between sex and love?,What is the difference between sex and love? l...
2,"Which is better, a laptop or a desktop?","Which is better, a laptop or a desktop? portab..."
3,"Which is better, Canon or Nikon?","Which is better, Canon or Nikon? ericsson cano..."
4,What are the best dish detergents?,What are the best dish detergents? dispersant ...
5,What are the best cities to live in?,What are the best cities to live in? township ...
6,What is the longest river in the U.S.?,What is the longest river in the U.S.? u.s kir...
7,"Which is healthiest: coffee, green tea or blac...","Which is healthiest: coffee, green tea or blac..."
8,What are the advantages and disadvantages of P...,What are the advantages and disadvantages of P...
9,Why is Linux better than Windows?,Why is Linux better than Windows? ppa nt4 wind...
10,How to sleep better?,How to sleep better? dozed hibernation


# Export topics to txt

In [46]:
def write_text(file_name, m_list):
    myfile = open(file_name+'.txt', 'w')
    for text in m_list:
        myfile.write(text + "\n\n")
write_text("shorted_topics_wb", shorted_topics)
write_text("expanded_topics_wb", expanded_topics)

# End similar words

In [85]:
def read_text(file_name):
    f = open(file_name+".txt", "r")
    return [e for e in f.read().split("\n\n") if len(e)!=0]
import_shored_topics = read_text("shorted_topics")
len(import_shored_topics)

50

In [88]:
def base_api(topic, size):
    url = 'https://www.chatnoir.eu/api/v1/_search'
    request_data = {
        "apikey": "67fac2d9-0f98-4c19-aab0-18c848bfa130",
        "query": topic,
        "size": size,
        "index": ["cw12"],
    }
    def chatnoir_req():
        try:
            response_=requests.post(url, data=request_data)
            response_.raise_for_status()
            return response_.json()
        except requests.exceptions.HTTPError:
            time.sleep(1)
            return chatnoir_req()    
    resp = chatnoir_req()
    return resp
def base_main(size, file_name):
    topics = read_text(file_name)
    print(topics[0])
    out = open("data_sense2vec/"+file_name+".run", "w")
    answers = []
    
    for topic in topics:
        print("Getting response for", topic)
        answers.append(base_api(topic, size))
        index=index+1
        
    topicId = 1
    for topic in answers:
        rank = 1
        for response in topic['results']:
            buffer = topicId, "Q0", response['trec_id'], rank, response['score'], "JackSparrowVanilla"
            out.write(" ".join(map(str, buffer)) + "\n")
            rank += 1
        topicId += 1

In [89]:
base_main(15, 'shorted_topics')

true sex sexual just deep massive actual big difference love only large real partner
Getting response for true sex sexual just deep massive actual big difference love only large real partner


NameError: name 'requests' is not defined

# End

In [17]:
import numpy as np
from spacy.lang.en.stop_words import STOP_WORDS

def get_similar_words(word, topn):
    
    ms = nlp.vocab.vectors.most_similar(np.asarray([nlp.vocab.vectors[nlp.vocab.strings[word]]]), n=topn)
    #print(len(ms))
    words = [nlp.vocab.strings[w] for w in ms[0][0]]
    print(words)
    distances = ms[2]
    #print(distances)
get_similar_words("Yahoo", topn=3)

['ALTAVISTA', 'Yahoo', 'MESSINGER']


In [ ]:
for topic in topics:
    doc = nlp(topic)
    print(topic)
    for token in doc:
        print(token.s2v_other_senses)
        #print(token.text, token.tag_)
        tags = ['CD',
                        'JJ','JJR','JJS',
                        'RB','RBR','RBS', 
                        'NN', 'NNS','NNP','NNPS', 
                        'VB']
        pos_tags = ['PROPN','VERB','NOUN','NUM']
        assert token in s2v
        if token.tag_ in tags or token.pos_ in pos_tags:
            try:
                print(token._.s2v_most_similar(3))
            except ValueError as err:
                print(err)
            
    print(40*"=")

# other approach

In [10]:
def most_similar(word, topn=5):
    word = nlp.vocab[str(word)]
    
    queries = [
      w for w in word.vocab 
      if w.is_lower == word.is_lower and w.prob >= -15 and np.count_nonzero(w.vector)
    ]
    print(queries)
    by_similarity = sorted(queries, key=lambda w: word.similarity(w), reverse=True)
    return [(w.lower_,w.similarity(word)) for w in by_similarity[:topn+1] if w.lower_ != word.lower_]

most_similar("dog", topn=3)

[]


[]

In [ ]:
assert doc[3:6].text == "natural language processing"
freq = doc[3:6]._.s2v_freq
vector = doc[3:6]._.s2v_vec
most_similar = doc[3:6]._.s2v_most_similar(3)

In [ ]:
most_similar

In [ ]:
'''
import spacy
from sense2vec import Sense2VecComponent
nlp = en_core_web_md.load()

s2v = Sense2VecComponent.from_disk('s2v_reddit_2015_md')
nlp.add_pipe(s2v)
'''

In [ ]:
doc = nlp("A sentence about natural language processing.")
assert doc[3].text == 'natural language processing'
freq = doc[3]._.s2v_freq
vector = doc[3]._.s2v_vec
most_similar = doc[3]._.s2v_most_similar(3)